In [1]:
#준비
import pandas as pd
import csv
import logging
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

In [2]:
#구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#csv 파일 위치한 경로 입력
import os
os.chdir("/content/drive/MyDrive/최종프로젝트")

In [4]:
#데이터 확인
justice = pd.read_csv('output1_0709.csv')
justice.head()

,사건번호,법원명,사건명,선고일자,판결결과,주문,전문,양형의 이유,유사판례1,내용1,유사판례2,내용2,유사판례3,내용3
0,2022고정296,춘천지방법원,건축법위반,2023. 1. 31.,벌금,피고인을 벌금 150만 원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우 1...,범 죄 사 실 피고인은 강원 홍천군 B 토지 소유자이다. 「국토의 계획 및 이용에 ...,NaN,전주지방법원 군산지원 2022. 9. 21. 선고 2020고정74 판결 [건축법위반...,"범 죄 사 실 누구든지 재해복구, 흥행, 전람회, 공사용 가설건축물 등 대통령령으로...",춘천지방법원 영월지원 2023. 10. 10. 선고 2023고정38 판결 [건축법위...,이 유 범 죄 사 실 누구든지 연면적의 합계가 100제곱미터 이하인 건축물을 건축하...,수원지방법원 성남지원 2023. 5. 12. 선고 2023고정48 판결 [개발제한구...,이 유 범 죄 사 실 누구든지 개발제한구역에서는 관할관청의 허가를 받지 아니하고 건...
1,2022고단1433,창원지방법원 마산지원,건조물침입,2023. 1. 31.,집행유예,"피고인을 징역 6개월에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 ...","범 죄 사 실 피고인은 창원시 의창구 B, C호에 본점을 두고 자동검사기 제조업 등...",1. 법률상 처단형의 범위: 징역 1개월~3년 2. 양형기준에 따른 권고형의 범위 ...,의정부지방법원 2021. 4. 5. 선고 2020고단4619 판결 [절도] \n,이 유 범 죄 사 실 피고인은 2018. 1. 19.경부터 같은 달 26.경 사이에...,창원지방법원 마산지원 2023. 11. 1. 선고 2023고단669 판결 [사기] \n,이 유 범 죄 사 실 피고인은 경남 함안군 B에 있는 ㈜C의 실제 운영자이다. 피고...,서울중앙지방법원 2023. 6. 1. 선고 2022고단6842 판결 [사기] \n,이 유 범 죄 사 실 피고인은 2017. 5. 22.경 수원시 장안구 B 소재 중고...
2,2022고단1316,춘천지방법원,산지관리법위반,2023. 1. 31.,벌금,[피고인 A] 피고인을 벌금 600만 원에 처한다. 피고인이 위 벌금을 납입하지 아...,"범 죄 사 실 피고인 주식회사 B는 인천 부평구 C, 4층 D호에 소재지를 두고 부...","무허가 산지일시사용이 이루어진 면적이 상당하나, 피고인들이 이 사건 범행을 모두 인...",대구지방법원 김천지원 2023. 1. 11. 선고 2022고단682 판결 [산지관리...,범 죄 사 실 피고인 A은 김천시 C에 있는 골재채취업 등을 목적으로 하는 B 주식...,수원지방법원 2023. 8. 10. 선고 2023고단1537 판결 [산지관리법위반] \n,"이 유 범 죄 사 실 피고인 A은 주식회사 B의 대표이사이고, 피고인 주식회사 B은...",광주지방법원 목포지원 2022. 4. 19. 선고 2021고정117 판결 [산업안전...,이 유 범 죄 사 실 피고인 주식회사 B는 성남시 분당구 C에서 통신업을 목적으로 ...
3,2022고단861,대전지방법원 홍성지원,절도,2023. 1. 31.,집행유예,"피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유...",범 죄 사 실 피고인은 피해자 B 주식회사(이하 '피해자 회사'라 한다)의 협력업...,"절취의 태양 및 횟수 등을 고려할 때, 죄책이 결코 가볍지 아니하나, 잘못을 인...",광주지방법원 목포지원 2023. 12. 8. 선고 2023고단836 판결 [야간건조...,이 유 범 죄 사 실 피고인은 2023. 6. 22. 01:30경부터 04:30경까...,광주지방법원 목포지원 2023. 2. 10. 선고 2022고단284 판결 [업무상횡...,범 죄 사 실 『2022고단284』 1. 업무상횡령 피고인은 2019. 8. 6.경...,서울서부지방법원 2024. 4. 4. 선고 2023고단1934 판결 [업무상횡령] \n,이 유 범 죄 사 실 피고인은 2019. 6. 1.경부터 2022. 10. 31.경...
4,2022노3065,대전지방법원,사기,2023. 1. 31.,징역,원심판결 중 피고사건 부분을 파기한다. 피고인을 징역 2년에 처한다.,"1. 이 법원의 심판범위 원심은 원심 배상신청인의 배상명령신청을 각하하였는바, 소송...",앞서 제3항에서 살펴본 제반 양형조건을 참작하여 주문과 같이 형을 정한다.,전주지방법원 2023. 10. 12. 선고 2023노865 판결 [사기] \n,"이 유 1. 이 법원의 심판범위 원심은 원심 배상신청인의 배상신청을 각하하였는데, ...",수원지방법원 2023. 11. 14. 선고 2023노3246 판결 [사기] \n,이 유 1. 이 법원의 심판범위 원심은 배상신청인들의 배상신청을 모두 각하하였다. ...,부산지방법원 2023. 12. 21. 선고 2023노3749 판결 [사기] \n,"이 유 1. 이 법원의 심판범위 원심은 배상신청인들의 배상신청을 각하하였는데, 배상..."


In [5]:
justice['주문'] = justice['주문'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
justice['전문'] = justice['전문'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
justice['양형의 이유'] = justice['양형의 이유'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
justice['유사판례1'] = justice['유사판례1'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
justice['내용1'] = justice['내용1'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
justice['유사판례2'] = justice['유사판례2'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
justice['내용2'] = justice['내용2'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
justice['유사판례3'] = justice['유사판례3'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
justice['내용3'] = justice['내용3'].str.replace(pat=r'[^\w]', repl=r'', regex=True)

In [24]:
justice.count()

사건번호      995
법원명       995
사건명       995
선고일자      995
판결결과      995
주문        995
전문        995
양형의 이유    915
유사판례1     995
내용1       995
유사판례2     995
내용2       995
유사판례3     995
내용3       995
dtype: int64

In [6]:
justice.tail()

,사건번호,법원명,사건명,선고일자,판결결과,주문,전문,양형의 이유,유사판례1,내용1,유사판례2,내용2,유사판례3,내용3
990,2023고합9,청주지방법원 충주지원,살인,2023. 5. 25.,징역,피고인을징역12년에처한다압수된가위1개증제1호를몰수한다,범죄사실기초사실피고인은피해자B여78세의배우자이다피고인은과거신장수술등으로인해건강상태가...,1법률상처단형의범위징역5년30년이하2양형기준에따른권고형의범위유형의결정살인범죄제2유형...,대전지방법원2022129선고2022고합305판결살인,이유범죄사실피고인은피해자B남83세와어렸을때부터이웃에서거주하면서알고지낸관계이다피고인은...,서울남부지방법원20231011선고2023고합276판결살인미수,이유범죄사실기초사실피고인은2005경배우자사망후서울금천구B아파트C호에서혼자거주하다가2...,수원지방법원안양지원202369선고2022고합246판결유사강간치상,이유범죄사실피고인은피해자B여65세의법률상배우자이다피고인은2022612300경안양시동...
991,2023고단270,부산지방법원,사기,2023. 5. 25.,징역,피고인을징역1년6월에처한다배상신청인들의신청을모두각하한다,범죄사실전화금융사기조직은중국또는기타국내수사기관의추적이어려운지역에서인터넷전화등을이용하...,피고인의범행은불특정다수의피해자를상대로조직적계획적지능적으로이루어지는전화금융사기에가담한...,광주지방법원목포지원2023511선고2022고단1280판결사기방조,이유범죄사실전화금융사기조직은중국또는기타국내수사기관의추적이어려운지역에서인터넷전화등을이...,대구지방법원2023823선고2022고단5192판결사기사문서위조위조사,이유범죄사실공모관계전화금융사기조직은중국또는기타국내수사기관의추적이어려운지역에서인터넷전...,춘천지방법원2023711선고2023고단104판결사기,이유범죄사실2023고단104전화금융사기조직은중국또는기타국내수사기관의추적이어려운지역에...
992,2023고단76,창원지방법원 밀양지원,사기,2023. 5. 25.,집행유예,피고인들을징역1년에각처한다다만피고인B에대하여는이판결확정일로부터3년간위형의집행을유예한...,범죄사실1피해자F에대한범행피고인들은채팅어플리케이션을통해알게된여성들에게마치자신이대기업...,1법률상처단형의범위징역1개월22년6개월이하2양형기준에따른권고형의범위유형의결정사기범죄...,인천지방법원부천지원2020129선고2020고단2541판결사기사기미수,이유범죄사실2020고단2541성명불상자는전기통신금융사기일명보이스피싱조직의총책으로불특...,서울북부지방법원2022810선고2022고단175판결사기배상명령신청,이유범죄사실2022고단175피고인은202117경서울동대문구P아파트Q호에서인터넷포털사...,대구지방법원서부지원202375선고2022고단2728판결사기,이유범죄사실2022고단2728피고인은2022421경대구달서구T에있는피고인의주거지에서...
993,2023노106,청주지방법원,사기,2023. 5. 25.,벌금,원심판결중피고사건부분을파기한다피고인을벌금1000만원에처한다피고인이위벌금을납입하지아니...,1이법원의심판범위가원심은원심배상신청인B의배상명령신청을인용하였는바유죄판결에대한상소가제...,NaN,서울중앙지방법원202329선고2022노1170판결사기,이유1이법원의심판범위원심은배상신청인의배상명령신청을각하하였는데소송촉진등에관한특례법제3...,대구지방법원2020522선고2020노945판결사기,1항소이유의요지양형부당원심이피고인에게선고한형징역6월은너무무거워서부당하다2판단가피고인...,청주지방법원2022929선고2022노614판결사기,이유1이법원의심판범위원심은배상신청인의배상명령신청을각하하였는바소송촉진등에관한특례법제3...
994,2023고단698,서울동부지방법원,사기,2023. 5. 25.,집행유예,피고인들을각징역4개월에처한다다만피고인들에대하여이판결확정일부터1년간위각형의집행을유예한...,범죄사실1피고인A피고인은20181210경불상지에서시누이亡D를통하여알게된피해자C에게내...,피고인들이자신의잘못을반성하고있는점피해자에게이자명목으로피고인A은1658만원피고인B은1...,서울중앙지방법원20231012선고2023고단4028판결컴퓨터등사용사기여신전문,이유범죄사실피고인들은자매이다1피고인A의단독범행가피고인은202212경불상의장소에서피해...,울산지방법원2022623선고2021고단3230판결업무상횡령배상명령신청,범죄사실피고인A피고인B은201874경부터2019629경까지울산울주군D에있는피해자인합...,춘천지방법원202354선고2022고단1463판결준사기식품위생법위반,이유범죄사실범죄전력피고인A는2022527춘천지방법원에서준사기죄등으로징역3년및벌금10...


In [7]:
import pandas as pd
justice['양형의 이유'].count()

915

In [8]:
justice.isnull().sum()

사건번호       0
법원명        0
사건명        0
선고일자       0
판결결과       0
주문         0
전문         0
양형의 이유    80
유사판례1      0
내용1        0
유사판례2      0
내용2        0
유사판례3      0
내용3        0
dtype: int64

In [9]:
print(justice['양형의 이유'].isna().sum())

80


In [22]:
# '양형의 이유' 결측값 행 제거
import pandas as pd
justice_altered = justice.dropna(axis=0)

In [23]:
justice_altered.count()

사건번호      915
법원명       915
사건명       915
선고일자      915
판결결과      915
주문        915
전문        915
양형의 이유    915
유사판례1     915
내용1       915
유사판례2     915
내용2       915
유사판례3     915
내용3       915
dtype: int64

In [27]:
justice_altered.to_csv('justice_altered.csv', index=False, encoding='utf-8-sig')